## Jieba分词Script

### jieba分词github：https://github.com/fxsjy/jieba

### 1、jieba分词原理
#### 1.1 基于规则是指根据一个已有的词典，采用前向最大匹配、后向最大匹配、双向最大匹配等人工设定的规则来进行分词。例如对于 “上海自来水来自海上” 这句话，使用前向最大匹配，即从前向后扫描，使分出来的词存在于词典中并且尽可能长，则可以得到 “上海 / 自来水 / 来自 / 海上”。
#### 1.2 基于统计是从大量人工标注语料中总结词的概率分布以及词之间的常用搭配，使用有监督学习训练分词模型。对于 “上海自来水来自海上” 这句话，一个最简单的统计分词想法是，尝试所有可能的分词方案，因为任何两个字之间，要么需要切分，要么无需切分。

### 2、jieba安装

python2下
```
pip install jieba
```

python3下
```
pip3 install jieba
```

### 3、jieba使用demo

In [2]:
import pandas as pd
import jieba

In [4]:
#数据读入
data = pd.read_csv('pointtag.csv')
jieba.load_userdict("fool_dic")
#加载停用词表
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]  
    return stopwords  
#对知识点短语分词
#IO 时间损耗，每次运行程序都会读入停用词表
def seg_keypoint(sentence):  
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist('stopwords.txt')  # 这里加载停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr  


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/_2/6z88r20s6xz6nh2npj1wm14c0000gn/T/jieba.cache
Loading model cost 1.436 seconds.
Prefix dict has been built succesfully.


In [6]:
#选取pointtag.csv中1000条数据进行测试
result = list()
for keyword in data['知识点'][0:1000]:
    tag = seg_keypoint(keyword)
    result.append(tag.strip())
result = pd.DataFrame(result)

In [7]:
result

,0
0,二元一次方程 定义 求 参数
1,二元一次方程 识别
2,二元一次方程 识别
3,二元一次方程 识别
4,二元一次方程 识别
5,二元一次方程组 解 解 二元一次方程组 叠加 减法
6,二元一次方程组 解 解 二元一次方程组 加 减消 元法
7,解 二元一次方程 中 参数 值
8,解 二元一次方程 中 参数 值
9,解 二元一次方程 中 参数 值


### 4、jieba分词的其他应用

#### 4.1关键词的提取

#### jieba 还实现了 TF-IDF 和 TextRank 这两种关键词提取算法提取关键词的前提是中文分词，所以这里也会使用到 jieba 自带的前缀词典和 IDF 权重词典。

In [10]:
import jieba.analyse

# 字符串前面加u表示使用unicode编码
content = u'中国特色社会主义是我们党领导的伟大事业，全面推进党的建设新的伟大工程，是这一伟大事业取得胜利的关键所在。党坚强有力，事业才能兴旺发达，国家才能繁荣稳定，人民才能幸福安康。党的十八大以来，我们党坚持党要管党、从严治党，凝心聚力、直击积弊、扶正祛邪，党的建设开创新局面，党风政风呈现新气象。习近平总书记围绕从严管党治党提出一系列新的重要思想，为全面推进党的建设新的伟大工程进一步指明了方向。'
# 第一个参数：待提取关键词的文本
# 第二个参数：返回关键词的数量，重要性从高到低排序
# 第三个参数：是否同时返回每个关键词的权重
# 第四个参数：词性过滤，为空表示不过滤，若提供则仅返回符合词性要求的关键词
keywords = jieba.analyse.extract_tags(content, topK=20, withWeight=True, allowPOS=())
# 访问提取结果
for item in keywords:
    # 分别为关键词和相应的权重
    print(item[0], item[1])

# 同样是四个参数，但allowPOS默认为('ns', 'n', 'vn', 'v')
# 即仅提取地名、名词、动名词、动词
keywords = jieba.analyse.textrank(content, topK=20, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v'))
# 访问提取结果
for item in keywords:
    # 分别为关键词和相应的权重
    print(item[0], item[1])

党的建设 0.47331204260459014
管党 0.3919595902590164
伟大工程 0.3771404058754098
伟大事业 0.3669713918327869
才能 0.26339384065180327
治党 0.22787996150819673
党要 0.1959797951295082
从严治党 0.1959797951295082
凝心 0.1959797951295082
聚力 0.1959797951295082
直击 0.1959797951295082
坚强有力 0.19013266490163933
扶正祛邪 0.19013266490163933
推进 0.18810840444327867
政风 0.18583161138524593
全面 0.18439437791967214
党风 0.17961047004590164
新气象 0.17267839052459016
兴旺发达 0.16782157386557378
习近平 0.1624867804165574
才能 1.0
管党 0.7999933934163805
全面 0.7325692441985737
社会主义 0.6327916888315029
围绕 0.60594603358887
总书记 0.5945625023471114
凝心 0.5840883789052874
政风 0.5792034335473362
新气象 0.5772168490112909
党风 0.5728262292165519
呈现 0.5700456186486299
推进 0.5548361394986431
方向 0.5150324602730256
指明 0.5113586590717408
治党 0.5062232626208965
局面 0.4744549207999055
聚力 0.46596165707522896
积弊 0.4646149902996275
直击 0.46314922535402286
国家 0.46179235227324805


### 5、jieba与其他工具比较

#### 参看文件中的img1和img2